## AF5365 - Applications of Computing and Technology in Accounting and Finance I
## Group Project - Source Code of Analysis

##### LendingClub Corporation (LC  US:NYSE)
LendingClub Corporation provides Internet financial services. The Company hosts an online financial community that brings together credit worthy borrowers and independent investors for their mutual benefits. LendingClub serves retail investors and borrowers in the United States.

### Part 1: Web Scraping

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# url for recent 10-K
url = 'https://www.sec.gov/Archives/edgar/data/1409970/000140997019000222/0001409970-19-000222.txt'

# remove the 'ACCESSION NUMBER' to get the base url
base_url = url.replace('/0001409970-19-000222.txt','')

In [3]:
# all the related filing can be found in the base_url
base_url

'https://www.sec.gov/Archives/edgar/data/1409970/000140997019000222'

In [4]:
# url for FilingSummary.xml
filing_summary_url = base_url + '/FilingSummary.xml'
filing_summary_url

'https://www.sec.gov/Archives/edgar/data/1409970/000140997019000222/FilingSummary.xml'

In [5]:
# 'FilingSummary.xml' contains information for the filing
summary_content = requests.get(filing_summary_url).content

soup = BeautifulSoup(summary_content, 'lxml')

reports = soup.find('myreports')

# create a list to store file information
all_reports = []

for report in reports.find_all('report')[:-1]:
    
    # create a dictionary contains information for 'short name' and 'url'
    report_dict = {}
    report_dict['short_name'] = report.shortname.text
    report_dict['url'] = base_url + '/' + report.htmlfilename.text
    
    all_reports.append(report_dict)

In [6]:
len(all_reports)

103

In [7]:
all_reports

[{'short_name': 'Document and Entity Information',
  'url': 'https://www.sec.gov/Archives/edgar/data/1409970/000140997019000222/R1.htm'},
 {'short_name': 'Consolidated Balance Sheets',
  'url': 'https://www.sec.gov/Archives/edgar/data/1409970/000140997019000222/R2.htm'},
 {'short_name': 'Consolidated Balance Sheets (Parenthetical)',
  'url': 'https://www.sec.gov/Archives/edgar/data/1409970/000140997019000222/R3.htm'},
 {'short_name': 'Consolidated Statements of Operations',
  'url': 'https://www.sec.gov/Archives/edgar/data/1409970/000140997019000222/R4.htm'},
 {'short_name': 'Consolidated Statements of Comprehensive Income (Loss)',
  'url': 'https://www.sec.gov/Archives/edgar/data/1409970/000140997019000222/R5.htm'},
 {'short_name': "Consolidated Statements of Changes in Stockholders' Equity",
  'url': 'https://www.sec.gov/Archives/edgar/data/1409970/000140997019000222/R6.htm'},
 {'short_name': 'Consolidated Statements of Cash Flows',
  'url': 'https://www.sec.gov/Archives/edgar/data/1

In [8]:
# we need the data in balance sheet and income statement
statement_list = ['Consolidated Balance Sheets','Consolidated Statements of Operations']

statement_reports = []

for report_dict in all_reports:
    if report_dict['short_name'] in statement_list:
        statement_reports.append(report_dict)

In [9]:
statement_reports

[{'short_name': 'Consolidated Balance Sheets',
  'url': 'https://www.sec.gov/Archives/edgar/data/1409970/000140997019000222/R2.htm'},
 {'short_name': 'Consolidated Statements of Operations',
  'url': 'https://www.sec.gov/Archives/edgar/data/1409970/000140997019000222/R4.htm'}]

In [10]:
# extract data in balance sheet
# create a dictionary to store the related data 
BS_data = {}
BS_data['headers'] = []
BS_data['data'] = []

In [11]:
# url for 'Consolidated Balance Sheets'
BS_url = statement_reports[0]['url']

BS_content = requests.get(BS_url).content

BS_soup = BeautifulSoup(BS_content, 'html.parser')

# use for loop to loop through all 'tr'
for index, row in enumerate(BS_soup.table.find_all('tr')):
    
    # in each 'tr', each 'td' represent data in a column
    cols = row.find_all('td')
    
    # if it is not header and not in bold, it is the data row
    # extract data in this row
    if (len(row.find_all('th')) == 0 and len(row.find_all('strong')) == 0):
        reg_row = [ele.text.strip() for ele in cols]
        BS_data['data'].append(reg_row)
    
    # extract the header information
    elif (len(row.find_all('th')) != 0):
        hed_row = [ele.text.strip() for ele in row.find_all('th')]
        BS_data['headers'].append(hed_row)
    
    else:
        pass

In [12]:
# create a dataframe for storing the data
BS_data_df = pd.DataFrame(BS_data['data'])

# drop the column which is irrelevant
BS_data_df = BS_data_df.drop(1, axis = 1)

# assign the 'headers' into the dataframe
BS_data_df.columns = BS_data['headers'][0]

# rename the column name
BS_data_df.rename({'Consolidated Balance Sheets - USD ($) $ in Thousands': 'Data Name'}, axis=1, inplace=True)

# drop the row which is irrelevant
BS_data_df.drop(BS_data_df.tail(3).index, inplace=True)

In [13]:
BS_data_df.tail()

,Data Name,"Dec. 31, 2018","Dec. 31, 2017"
24,Accumulated other comprehensive income (loss),157,(5)
25,Total LendingClub stockholders’ equity,"869,201","922,495"
26,Noncontrolling interests,"1,780","5,262"
27,Total equity,"870,981","927,757"
28,Total liabilities and equity,"$ 3,819,527","$ 4,640,831"


In [14]:
# extract data in income statement
# create a dictionary to store the related data 
IS_data = {}
IS_data['headers'] = []
IS_data['data'] = []

In [15]:
# url for 'Consolidated Statements of Operations'
IS_url = statement_reports[1]['url']

IS_content = requests.get(IS_url).content

IS_soup = BeautifulSoup(IS_content, 'html.parser')

# use for loop to loop through all 'tr'
for index, row in enumerate(IS_soup.table.find_all('tr')):
    
    # in each 'tr', each 'td' represent data in a column
    cols = row.find_all('td')
    
    # if it is not header and not in bold, it is the data row
    # extract data in this row
    if (len(row.find_all('th')) == 0 and len(row.find_all('strong')) == 0):
        reg_row = [ele.text.strip() for ele in cols]
        IS_data['data'].append(reg_row)
    
    # extract the header information
    elif (len(row.find_all('th')) != 0):
        hed_row = [ele.text.strip() for ele in row.find_all('th')]
        IS_data['headers'].append(hed_row)
        
    else:
        pass

In [16]:
# create a dataframe for storing the data
IS_data_df = pd.DataFrame(IS_data['data'])

# create the headers list
IS_data_headers = []
IS_data_headers.append('Data Name')
for item in IS_data['headers'][1]:
    IS_data_headers.append(item)

# assign the 'headers' into the dataframe
IS_data_df.columns = IS_data_headers

# drop the row which is irrelevant
IS_data_df.drop(IS_data_df.tail(4).index, inplace=True)

# get data of 'Dec. 31, 2018' and 'Dec. 31, 2017' under '12 Months Ended'
IS_data_df2 = IS_data_df.iloc[:,[0,9,10]]

In [17]:
IS_data_df2.tail()

,Data Name,"Dec. 31, 2018","Dec. 31, 2017"
19,LendingClub net loss,"$ (128,308,000)","$ (153,835,000)"
20,Basic net loss per share attributable to commo...,$ (0.30),$ (0.38)
21,Diluted net loss per share attributable to com...,$ (0.30),$ (0.38)
22,Weighted-average common shares - Basic (shares),"422,917,308","408,995,947"
23,Weighted-average common shares - Diluted (shares),"422,917,308","408,995,947"


In [18]:
# create a dataframe to store all extracted data
all_data_df = pd.DataFrame()

# concatenate two dataframe 'BS_data_df' and 'IS_data_df2'
all_data_df = pd.concat([BS_data_df, IS_data_df2], ignore_index=True)

# formatting the number in all_data_df
all_data_df2 = all_data_df.iloc[:,1:3].replace({'[\$, )]':'', '[(]':'-', '': 'NaN'}, regex = True)

all_data_df3 = pd.concat([all_data_df.iloc[:,0], all_data_df2], axis = 1)

# write dataframe to csv
all_data_df3.to_csv('data.csv', index=False)

### Part 2: Calculation

In [19]:
df = pd.read_csv('data.csv', index_col = 0)

In [20]:
df.head()

,"Dec. 31, 2018","Dec. 31, 2017"
Data Name,,
Cash and cash equivalents,372974.0,401719.0
Restricted cash (1),271084.0,242570.0
"Securities available for sale (includes $53,611 and $0 pledged as collateral at fair value, respectively)",170469.0,117573.0
Loans held for investment at fair value (1),1883251.0,2932325.0
Loans held for investment by the Company at fair value (1),2583.0,361230.0


In [21]:
# change df to dictionary as df_dict
df_dict = df.to_dict()

In [22]:
# create a dictionary to store data for calculating the ratio
ratio_data_dict = {}

# for 'Current Assets'
ratio_data_dict['Current Assets'] = df_dict['Dec. 31, 2018']['Cash and cash equivalents'] + df_dict['Dec. 31, 2018']['Restricted cash (1)'] + df_dict['Dec. 31, 2018']['Securities available for sale (includes $53,611 and $0 pledged as collateral at fair value, respectively)'] + df_dict['Dec. 31, 2018']['Loans held for investment at fair value (1)'] + df_dict['Dec. 31, 2018']['Loans held for investment by the Company at fair value (1)'] + df_dict['Dec. 31, 2018']['Loans held for sale by the Company at fair value (1)'] + df_dict['Dec. 31, 2018']['Accrued interest receivable (1)'] 

# for 'Quick Assets'
ratio_data_dict['Quick Assets'] = df_dict['Dec. 31, 2018']['Cash and cash equivalents'] + df_dict['Dec. 31, 2018']['Restricted cash (1)'] + df_dict['Dec. 31, 2018']['Securities available for sale (includes $53,611 and $0 pledged as collateral at fair value, respectively)'] + df_dict['Dec. 31, 2018']['Loans held for investment at fair value (1)'] + df_dict['Dec. 31, 2018']['Loans held for investment by the Company at fair value (1)'] + df_dict['Dec. 31, 2018']['Loans held for sale by the Company at fair value (1)'] + df_dict['Dec. 31, 2018']['Accrued interest receivable (1)'] 

# for 'Cash + Marketable Securities'
ratio_data_dict['Cash + Marketable Securities'] = df_dict['Dec. 31, 2018']['Cash and cash equivalents'] + df_dict['Dec. 31, 2018']['Restricted cash (1)'] + df_dict['Dec. 31, 2018']['Securities available for sale (includes $53,611 and $0 pledged as collateral at fair value, respectively)']                    

# for 'Average Inventory'
# there is no related data for 'Inventory', so 'None' is assigned here
ratio_data_dict['Average Inventory'] = None

# for 'Total Asset'
ratio_data_dict['Total Asset'] = df_dict['Dec. 31, 2018']['Total assets']        

# for 'Current Liabilities'
ratio_data_dict['Current Liabilities'] = df_dict['Dec. 31, 2018']['Accounts payable'] + df_dict['Dec. 31, 2018']['Accrued interest payable (1)'] + df_dict['Dec. 31, 2018']['Accrued expenses and other liabilities (1)']                    

# for 'Total Debt'
ratio_data_dict['Total Debt'] = df_dict['Dec. 31, 2018']['Payable to investors'] + df_dict['Dec. 31, 2018']['Notes, certificates and secured borrowings at fair value (1)'] + df_dict['Dec. 31, 2018']['Payable to securitization note and residual certificate holders (includes $0 and$1,479 at fair value, respectively) (1)'] + df_dict['Dec. 31, 2018']['Credit facilities and securities sold under repurchase agreements (1)']                     

# for 'Total Liabilities'
ratio_data_dict['Total Liabilities'] = df_dict['Dec. 31, 2018']['Total liabilities']

# for 'Total Equity'
ratio_data_dict['Total Equity'] = df_dict['Dec. 31, 2018']['Total equity']

# for 'Revenue'
ratio_data_dict['Revenue'] = df_dict['Dec. 31, 2018']['Net revenue, fees'] 

# for 'Cost of Goods Sold'
ratio_data_dict['Cost of Goods Sold'] = df_dict['Dec. 31, 2018']['Gain (Loss) on sales of loans']

# for 'Net Revenue'
ratio_data_dict['Net Revenue'] = df_dict['Dec. 31, 2018']['Total net revenue']

# for 'Operating Expenses'
ratio_data_dict['Operating Expenses'] = df_dict['Dec. 31, 2018']['Total operating expenses']

# for 'Non Operating Income'
# there is no related data for 'Non Operating Income', so 'None' is assigned here
ratio_data_dict['Non Operating Income'] = None

# for 'Earnings Before Interest and Taxes'
ratio_data_dict['Earnings Before Interest and Taxes'] = df_dict['Dec. 31, 2018']['Loss before income tax expense']

# for 'Interest Expense'
ratio_data_dict['Interest Expense'] = df_dict['Dec. 31, 2018']['Interest expense']

# for 'Net Income'
ratio_data_dict['Net Income'] = df_dict['Dec. 31, 2018']['LendingClub net loss']

# for 'Preferred Dividend'
# there is no related data for 'Preferred Dividend', so 'None' is assigned here
ratio_data_dict['Preferred Dividend'] = None

# for 'Weighted Average Outstanding Shares'
ratio_data_dict['Weighted Average Outstanding Shares'] = df_dict['Dec. 31, 2018']['Weighted-average common shares - Basic (shares)']


In [23]:
ratio_data_dict

{'Current Assets': 3562637.0,
 'Quick Assets': 3562637.0,
 'Cash + Marketable Securities': 814527.0,
 'Average Inventory': None,
 'Total Asset': 3819527.0,
 'Current Liabilities': 178463.0,
 'Total Debt': 2770083.0,
 'Total Liabilities': 2948546.0,
 'Total Equity': 870981.0,
 'Revenue': 530587000.0,
 'Cost of Goods Sold': 45979000.0,
 'Net Revenue': 694812000.0,
 'Operating Expenses': 822922000.0,
 'Non Operating Income': None,
 'Earnings Before Interest and Taxes': -128110000.0,
 'Interest Expense': -385605000.0,
 'Net Income': -128308000.0,
 'Preferred Dividend': None,
 'Weighted Average Outstanding Shares': 422917308.0}

In [24]:
# create a dictionary to store the result for ratio
ratio_result_dict = {}

# Calculate liquidity ratio
ratio_result_dict['Current Ratio'] = ratio_data_dict['Current Assets'] / ratio_data_dict['Current Liabilities'] 
ratio_result_dict['Quick Ratio'] = ratio_data_dict['Quick Assets'] / ratio_data_dict['Current Liabilities']
ratio_result_dict['Cash Ratio'] = ratio_data_dict['Cash + Marketable Securities'] / ratio_data_dict['Current Liabilities']

# calculate profitability ratio
ratio_result_dict['Gross Profit Margin'] = (ratio_data_dict['Revenue'] - ratio_data_dict['Cost of Goods Sold']) / ratio_data_dict['Revenue']
ratio_result_dict['Operating Margin'] = (ratio_data_dict['Revenue'] - ratio_data_dict['Cost of Goods Sold'] - ratio_data_dict['Operating Expenses']) / ratio_data_dict['Revenue']
ratio_result_dict['Profit Margin'] = ratio_data_dict['Net Income'] / ratio_data_dict['Revenue']
ratio_result_dict['Earnings per Share'] = ratio_data_dict['Net Income'] / ratio_data_dict['Weighted Average Outstanding Shares']

# calculate leverage ratio
ratio_result_dict['Debt to Equity Ratio'] = ratio_data_dict['Total Debt'] / ratio_data_dict['Total Equity']
ratio_result_dict['Debt to Asset Ratio'] = ratio_data_dict['Total Debt'] / ratio_data_dict['Total Asset']
ratio_result_dict['Debt Ratio'] = ratio_data_dict['Total Liabilities'] / ratio_data_dict['Total Asset']
ratio_result_dict['Interest Coverage Ratio'] = ratio_data_dict['Earnings Before Interest and Taxes'] / ratio_data_dict['Interest Expense']

# calculate activity ratio / efficiency ratio
ratio_result_dict['Receivable Ratio'] = 'Not Required'
ratio_result_dict['Inventory Turnover Ratio'] = 'Not Applicable'
ratio_result_dict['Asset Turnover Ratio'] = ratio_data_dict['Net Revenue'] / ratio_data_dict['Total Asset']

In [25]:
# create the list for the index base on four ratio category
index_col = []

ratio_cat = ['Liquidity Ratio', 'Profitability Ratio', 'Leverage Ratio', 'Activity Ratio / Efficiency Ratio']

for item in ratio_cat:
    if (item == 'Liquidity Ratio' or item == 'Activity Ratio / Efficiency Ratio'):
        # repeat the item 3 times
        index_col.extend([item for i in range(3)])
    
    else:
        # repeat the item 4 times
        index_col.extend([item for i in range(4)])

# change the list to dataframe
index_col_df = pd.DataFrame(index_col, columns = ['Category'])

In [26]:
# change the dictionary to dataframe
# the ratio result can then present in a dataframe
result_df = pd.DataFrame.from_dict(ratio_result_dict, orient='index', columns = ['Result'])

result_df = result_df.reset_index()

result_df.rename(columns = {'index':'Ratio Name'}, inplace = True)

# concatenate two dataframe
result_df = pd.concat([result_df, index_col_df], axis = 1)

# set index in multi-index 
# create the final result dataframe
final_result_df = result_df.set_index(['Category', 'Ratio Name'])

In [27]:
final_result_df

Result
Category                          Ratio Name                              
Liquidity Ratio                   Current Ratio                    19.9629
                                  Quick Ratio                      19.9629
                                  Cash Ratio                       4.56412
Profitability Ratio               Gross Profit Margin             0.913343
                                  Operating Margin               -0.637622
                                  Profit Margin                  -0.241823
                                  Earnings per Share             -0.303388
Leverage Ratio                    Debt to Equity Ratio             3.18042
                                  Debt to Asset Ratio             0.725242
                                  Debt Ratio                      0.771966
                                  Interest Coverage Ratio         0.332231
Activity Ratio / Efficiency Ratio Receivable Ratio            Not Required
                                  Inventory Turnover Ratio  Not Applicable
                                  Asset Turnover Ratio              181.91